# 📊 Mexico Electricity Demand Analysis (2023–2025)

**Author:** Diego Ramírez  
**Date:** July 2025  
**Description:**  
This notebook explores hourly electricity demand patterns in Mexico using public data. We analyze temporal trends, visualize seasonality, and extract actionable insights useful for forecasting and grid planning.

**Tools:** pandas, matplotlib, seaborn, numpy  
**Data:** SVs downloaded from CENACE: https://www.cenace.gob.mx/Paginas/SIM/Reportes/EstimacionDemandaReal.aspx

## 0. ⚙️ Imports

In [1]:
import os
import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm

## 1. 📥 Data Loading

We begin by loading the full dataset from CSV files, parsing date columns, and checking the overall shape and structure.

In [2]:
# Files have 8 rows at the top that we must skip, since the dataset starts at row 9.

df_sample = pd.read_csv("./data/raw_data/Demanda Real Balance_0_v3 Dia Operacion 2023-01-01 v2023 01 15_12 25 01.csv", skiprows=8)
df_sample.head()

,Sistema,Area,Hora,Generacion (MWh),Importacion Total (MWh),Exportacion Total (MWh),Intercambio neto entre Gerencias (MWh),Estimacion de Demanda por Balance (MWh)
0,BCA,BCA,1,1036.03252,22.10174,20.97918,---,1037.15508
1,BCA,BCA,2,1027.37909,43.79153,44.91889,---,1026.25173
2,BCA,BCA,3,1042.72685,51.27296,46.05216,---,1047.94764
3,BCA,BCA,4,1022.79761,55.63775,55.51087,---,1022.92449
4,BCA,BCA,5,1001.48521,65.39402,70.70259,---,996.17664


In [3]:
df_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216 entries, 0 to 215
Data columns (total 8 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Sistema                                    216 non-null    object 
 1    Area                                      216 non-null    object 
 2    Hora                                      216 non-null    int64  
 3    Generacion (MWh)                          216 non-null    float64
 4    Importacion Total (MWh)                   216 non-null    float64
 5    Exportacion Total (MWh)                   216 non-null    float64
 6    Intercambio neto entre Gerencias (MWh)    216 non-null    object 
 7    Estimacion de Demanda por Balance (MWh)   216 non-null    float64
dtypes: float64(4), int64(1), object(3)
memory usage: 13.6+ KB


Data is clean and structured, but there are some initial concerns to consider for when the full data is consolidated into one big dataset:
* No date column, will probably have to add it via the file name.
* Change "---" to np.nan in Intercambio neto entre Gerencias

In [4]:
DATA_FOLDER = "./data/raw_data/"

csv_files = glob.glob(os.path.join(DATA_FOLDER, "*.csv"))
print(f"Found {len(csv_files)} CSV files")

Found 3548 CSV files


In [6]:
df_sample = pd.read_csv(csv_files[0], skiprows=8)
df_sample["file_name"] = os.path.basename(csv_files[0])
df_sample.head()

,Sistema,Area,Hora,Generacion (MWh),Importacion Total (MWh),Exportacion Total (MWh),Intercambio neto entre Gerencias (MWh),Estimacion de Demanda por Balance (MWh),file_name
0,BCA,BCA,1,1036.03252,22.10174,20.97918,---,1037.15508,Demanda Real Balance_0_v3 Dia Operacion 2023-0...
1,BCA,BCA,2,1027.37909,43.79153,44.91889,---,1026.25173,Demanda Real Balance_0_v3 Dia Operacion 2023-0...
2,BCA,BCA,3,1042.72685,51.27296,46.05216,---,1047.94764,Demanda Real Balance_0_v3 Dia Operacion 2023-0...
3,BCA,BCA,4,1022.79761,55.63775,55.51087,---,1022.92449,Demanda Real Balance_0_v3 Dia Operacion 2023-0...
4,BCA,BCA,5,1001.48521,65.39402,70.70259,---,996.17664,Demanda Real Balance_0_v3 Dia Operacion 2023-0...


In [7]:
def add_file_name(path: str) -> pd.DataFrame:
    # Reads the csv file from the path and adds a coulumn with the file name.
    df = pd.read_csv(path, skiprows=8)
    df["file_name"] = os.path.basename(path)

    return df

all_dataframes = []

for file in tqdm(csv_files, desc="Reading CSVs"):
    try:
        df = add_file_name(file)
        all_dataframes.append(df)
    except Exception as e:
        print(f"Failed to read {file}: {e}")

Reading CSVs:   0%|          | 0/3548 [00:00<?, ?it/s]

In [8]:
full_df = pd.concat(all_dataframes, ignore_index=True)
full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 766364 entries, 0 to 766363
Data columns (total 9 columns):
 #   Column                                     Non-Null Count   Dtype  
---  ------                                     --------------   -----  
 0   Sistema                                    766364 non-null  object 
 1    Area                                      766364 non-null  object 
 2    Hora                                      766364 non-null  int64  
 3    Generacion (MWh)                          766364 non-null  float64
 4    Importacion Total (MWh)                   766364 non-null  float64
 5    Exportacion Total (MWh)                   766364 non-null  float64
 6    Intercambio neto entre Gerencias (MWh)    766364 non-null  object 
 7    Estimacion de Demanda por Balance (MWh)   766364 non-null  float64
 8   file_name                                  766364 non-null  object 
dtypes: float64(4), int64(1), object(4)
memory usage: 52.6+ MB


In [10]:
full_df.columns

Index(['Sistema', ' Area', ' Hora', ' Generacion (MWh)',
       ' Importacion Total (MWh)', ' Exportacion Total (MWh)',
       ' Intercambio neto entre Gerencias (MWh)',
       ' Estimacion de Demanda por Balance (MWh) ', 'file_name'],
      dtype='object')

In [25]:
full_df.rename(columns={
    "Sistema": "System",
    " Area": "Area",
    " Hora": "Hour",
    " Generacion (MWh)": "Generation (MWh)",
    " Importacion Total (MWh)": "Total Imports (MWh)",
    " Exportacion Total (MWh)": "Total Exports (MWh)",
    " Intercambio neto entre Gerencias (MWh)": "Net Exchange Between Regional Control Centers (MWh)",
    " Estimacion de Demanda por Balance (MWh) ": "Estimated Demand by Energy Balance (MWh)",},
              inplace=True)

In [26]:
full_df.columns

Index(['System', 'Area', 'Hour', 'Generation (MWh)', 'Total Imports (MWh)',
       'Total Exports (MWh)',
       'Net Exchange Between Regional Control Centers (MWh)',
       'Estimated Demand by Energy Balance (MWh)', 'file_name',
       'Operation Date'],
      dtype='object')

In [27]:
full_df.head()

,System,Area,Hour,Generation (MWh),Total Imports (MWh),Total Exports (MWh),Net Exchange Between Regional Control Centers (MWh),Estimated Demand by Energy Balance (MWh),file_name,Operation Date
0,BCA,BCA,1,1036.03252,22.10174,20.97918,---,1037.15508,Demanda Real Balance_0_v3 Dia Operacion 2023-0...,2023-01-01
1,BCA,BCA,2,1027.37909,43.79153,44.91889,---,1026.25173,Demanda Real Balance_0_v3 Dia Operacion 2023-0...,2023-01-01
2,BCA,BCA,3,1042.72685,51.27296,46.05216,---,1047.94764,Demanda Real Balance_0_v3 Dia Operacion 2023-0...,2023-01-01
3,BCA,BCA,4,1022.79761,55.63775,55.51087,---,1022.92449,Demanda Real Balance_0_v3 Dia Operacion 2023-0...,2023-01-01
4,BCA,BCA,5,1001.48521,65.39402,70.70259,---,996.17664,Demanda Real Balance_0_v3 Dia Operacion 2023-0...,2023-01-01


In [20]:
full_df["file_name"]

0         Demanda Real Balance_0_v3 Dia Operacion 2023-0...
1         Demanda Real Balance_0_v3 Dia Operacion 2023-0...
2         Demanda Real Balance_0_v3 Dia Operacion 2023-0...
3         Demanda Real Balance_0_v3 Dia Operacion 2023-0...
4         Demanda Real Balance_0_v3 Dia Operacion 2023-0...
                                ...                        
766359    Demanda Real Balance_4_v3 Dia Operacion 2023-0...
766360    Demanda Real Balance_4_v3 Dia Operacion 2023-0...
766361    Demanda Real Balance_4_v3 Dia Operacion 2023-0...
766362    Demanda Real Balance_4_v3 Dia Operacion 2023-0...
766363    Demanda Real Balance_4_v3 Dia Operacion 2023-0...
Name: file_name, Length: 766364, dtype: object

In [23]:
full_df["Operation Date"] = full_df["file_name"].str.extract(r"Dia Operacion (\d{4}-\d{2}-\d{2})")
full_df["Operation Date"] = pd.to_datetime(full_df["Operation Date"])

In [28]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 766364 entries, 0 to 766363
Data columns (total 10 columns):
 #   Column                                               Non-Null Count   Dtype         
---  ------                                               --------------   -----         
 0   System                                               766364 non-null  object        
 1   Area                                                 766364 non-null  object        
 2   Hour                                                 766364 non-null  int64         
 3   Generation (MWh)                                     766364 non-null  float64       
 4   Total Imports (MWh)                                  766364 non-null  float64       
 5   Total Exports (MWh)                                  766364 non-null  float64       
 6   Net Exchange Between Regional Control Centers (MWh)  766364 non-null  object        
 7   Estimated Demand by Energy Balance (MWh)             766364 non-null  floa

In [36]:
full_df["Net Exchange Between Regional Control Centers (MWh)"] = (
    full_df["Net Exchange Between Regional Control Centers (MWh)"].replace({"               ---": np.nan})
)
full_df.head()

,System,Area,Hour,Generation (MWh),Total Imports (MWh),Total Exports (MWh),Net Exchange Between Regional Control Centers (MWh),Estimated Demand by Energy Balance (MWh),file_name,Operation Date
0,BCA,BCA,1,1036.03252,22.10174,20.97918,NaN,1037.15508,Demanda Real Balance_0_v3 Dia Operacion 2023-0...,2023-01-01
1,BCA,BCA,2,1027.37909,43.79153,44.91889,NaN,1026.25173,Demanda Real Balance_0_v3 Dia Operacion 2023-0...,2023-01-01
2,BCA,BCA,3,1042.72685,51.27296,46.05216,NaN,1047.94764,Demanda Real Balance_0_v3 Dia Operacion 2023-0...,2023-01-01
3,BCA,BCA,4,1022.79761,55.63775,55.51087,NaN,1022.92449,Demanda Real Balance_0_v3 Dia Operacion 2023-0...,2023-01-01
4,BCA,BCA,5,1001.48521,65.39402,70.70259,NaN,996.17664,Demanda Real Balance_0_v3 Dia Operacion 2023-0...,2023-01-01


In [38]:
full_df["Net Exchange Between Regional Control Centers (MWh)"].unique()

array([nan, '        1400.85174', '        1384.45808', ...,
       '         -2530.875', '          -2530.49', '         -2357.214'],
      dtype=object)

In [39]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 766364 entries, 0 to 766363
Data columns (total 10 columns):
 #   Column                                               Non-Null Count   Dtype         
---  ------                                               --------------   -----         
 0   System                                               766364 non-null  object        
 1   Area                                                 766364 non-null  object        
 2   Hour                                                 766364 non-null  int64         
 3   Generation (MWh)                                     766364 non-null  float64       
 4   Total Imports (MWh)                                  766364 non-null  float64       
 5   Total Exports (MWh)                                  766364 non-null  float64       
 6   Net Exchange Between Regional Control Centers (MWh)  595896 non-null  object        
 7   Estimated Demand by Energy Balance (MWh)             766364 non-null  floa

In [41]:
full_df["Net Exchange Between Regional Control Centers (MWh)"] = (
    full_df["Net Exchange Between Regional Control Centers (MWh)"].str.strip()
)
full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 766364 entries, 0 to 766363
Data columns (total 10 columns):
 #   Column                                               Non-Null Count   Dtype         
---  ------                                               --------------   -----         
 0   System                                               766364 non-null  object        
 1   Area                                                 766364 non-null  object        
 2   Hour                                                 766364 non-null  int64         
 3   Generation (MWh)                                     766364 non-null  float64       
 4   Total Imports (MWh)                                  766364 non-null  float64       
 5   Total Exports (MWh)                                  766364 non-null  float64       
 6   Net Exchange Between Regional Control Centers (MWh)  595896 non-null  object        
 7   Estimated Demand by Energy Balance (MWh)             766364 non-null  floa

In [46]:
full_df["Net Exchange Between Regional Control Centers (MWh)"] = full_df["Net Exchange Between Regional Control Centers (MWh)"].astype(float)
full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 766364 entries, 0 to 766363
Data columns (total 10 columns):
 #   Column                                               Non-Null Count   Dtype         
---  ------                                               --------------   -----         
 0   System                                               766364 non-null  object        
 1   Area                                                 766364 non-null  object        
 2   Hour                                                 766364 non-null  int64         
 3   Generation (MWh)                                     766364 non-null  float64       
 4   Total Imports (MWh)                                  766364 non-null  float64       
 5   Total Exports (MWh)                                  766364 non-null  float64       
 6   Net Exchange Between Regional Control Centers (MWh)  595896 non-null  float64       
 7   Estimated Demand by Energy Balance (MWh)             766364 non-null  floa

In [48]:
full_df["Hour"].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25], dtype=int64)

In [49]:
full_df[full_df["Hour"] == 25]

,System,Area,Hour,Generation (MWh),Total Imports (MWh),Total Exports (MWh),Net Exchange Between Regional Control Centers (MWh),Estimated Demand by Energy Balance (MWh),file_name,Operation Date
66551,BCA,BCA,25,1266.74041,19.94526,33.50452,NaN,1253.18115,Demanda Real Balance_0_v3 Dia Operacion 2023-1...,2023-11-05
145175,BCA,BCA,25,1223.65880,112.93366,117.25632,NaN,1219.33614,Demanda Real Balance_0_v3 Dia Operacion 2024-1...,2024-11-03
262462,BCA,BCA,25,1266.74041,19.94526,33.50452,NaN,1253.18115,Demanda Real Balance_1_v3 Dia Operacion 2023-1...,2023-11-05
341086,BCA,BCA,25,1227.70877,112.93366,117.25632,NaN,1223.38611,Demanda Real Balance_1_v3 Dia Operacion 2024-1...,2024-11-03
449301,BCA,BCA,25,1266.74041,19.94526,33.50452,NaN,1253.18115,Demanda Real Balance_2_v3 Dia Operacion 2023-1...,2023-11-05
527925,BCA,BCA,25,1227.70877,112.93366,117.25632,NaN,1223.38611,Demanda Real Balance_2_v3 Dia Operacion 2024-1...,2024-11-03
624260,BCA,BCA,25,1266.74041,19.94526,33.50452,NaN,1253.18115,Demanda Real Balance_3_v3 Dia Operacion 2023-1...,2023-11-05
702884,BCA,BCA,25,1227.70877,112.93366,117.25632,NaN,1223.38611,Demanda Real Balance_3_v3 Dia Operacion 2024-1...,2024-11-03


In [51]:
full_df[full_df["Hour"] == 25].count()

System                                                 8
Area                                                   8
Hour                                                   8
Generation (MWh)                                       8
Total Imports (MWh)                                    8
Total Exports (MWh)                                    8
Net Exchange Between Regional Control Centers (MWh)    0
Estimated Demand by Energy Balance (MWh)               8
file_name                                              8
Operation Date                                         8
dtype: int64

In [79]:
full_df.groupby("Operation Date")["Hour"].nunique().unique()

array([24, 25], dtype=int64)

In [81]:
full_df.columns

Index(['System', 'Area', 'Hour', 'Generation (MWh)', 'Total Imports (MWh)',
       'Total Exports (MWh)',
       'Net Exchange Between Regional Control Centers (MWh)',
       'Estimated Demand by Energy Balance (MWh)', 'file_name',
       'Operation Date', 'Datetime'],
      dtype='object')

In [83]:
check_for_duplicates_col = ['System', 'Area', 'Hour', 'Generation (MWh)', 'Total Imports (MWh)',
       'Total Exports (MWh)',
       'Net Exchange Between Regional Control Centers (MWh)',
       'Estimated Demand by Energy Balance (MWh)',
       'Operation Date', 'Datetime']

In [84]:
full_df = full_df[~full_df.duplicated(subset=check_for_duplicates_col)]

In [76]:
full_df["Datetime"] = full_df["Operation Date"] + pd.to_timedelta(full_df["Hour"], unit="h")

In [85]:
full_df.head()

,System,Area,Hour,Generation (MWh),Total Imports (MWh),Total Exports (MWh),Net Exchange Between Regional Control Centers (MWh),Estimated Demand by Energy Balance (MWh),file_name,Operation Date,Datetime
0,BCA,BCA,1,1036.03252,22.10174,20.97918,NaN,1037.15508,Demanda Real Balance_0_v3 Dia Operacion 2023-0...,2023-01-01,2023-01-01 01:00:00
1,BCA,BCA,2,1027.37909,43.79153,44.91889,NaN,1026.25173,Demanda Real Balance_0_v3 Dia Operacion 2023-0...,2023-01-01,2023-01-01 02:00:00
2,BCA,BCA,3,1042.72685,51.27296,46.05216,NaN,1047.94764,Demanda Real Balance_0_v3 Dia Operacion 2023-0...,2023-01-01,2023-01-01 03:00:00
3,BCA,BCA,4,1022.79761,55.63775,55.51087,NaN,1022.92449,Demanda Real Balance_0_v3 Dia Operacion 2023-0...,2023-01-01,2023-01-01 04:00:00
4,BCA,BCA,5,1001.48521,65.39402,70.70259,NaN,996.17664,Demanda Real Balance_0_v3 Dia Operacion 2023-0...,2023-01-01,2023-01-01 05:00:00


In [87]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 437671 entries, 0 to 750035
Data columns (total 11 columns):
 #   Column                                               Non-Null Count   Dtype         
---  ------                                               --------------   -----         
 0   System                                               437671 non-null  object        
 1   Area                                                 437671 non-null  object        
 2   Hour                                                 437671 non-null  int64         
 3   Generation (MWh)                                     437671 non-null  float64       
 4   Total Imports (MWh)                                  437671 non-null  float64       
 5   Total Exports (MWh)                                  437671 non-null  float64       
 6   Net Exchange Between Regional Control Centers (MWh)  357212 non-null  float64       
 7   Estimated Demand by Energy Balance (MWh)             437671 non-null  float64  

In [89]:
full_df.reset_index(inplace=True)
full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 437671 entries, 0 to 437670
Data columns (total 13 columns):
 #   Column                                               Non-Null Count   Dtype         
---  ------                                               --------------   -----         
 0   level_0                                              437671 non-null  int64         
 1   index                                                437671 non-null  int64         
 2   System                                               437671 non-null  object        
 3   Area                                                 437671 non-null  object        
 4   Hour                                                 437671 non-null  int64         
 5   Generation (MWh)                                     437671 non-null  float64       
 6   Total Imports (MWh)                                  437671 non-null  float64       
 7   Total Exports (MWh)                                  437671 non-null  floa

In [90]:
full_df.to_csv("./data/processed_data/consolidated_data.csv")

## 2. 🧹 Data Cleaning

This section handles:
- Missing values
- Incorrect data types
- Duplicates (if any)
- Parsing datetime columns correctly

We'll also inspect and clean column names for consistency.

## 3. 🧮 Feature Engineering

We'll create new columns to support analysis:
- Hour, Day, Month, Weekday from timestamps
- Weekend vs Weekday
- Peak vs Off-Peak labeling (optional)

This makes it easier to explore time-based demand patterns.

## 4. 📈 Exploratory Data Analysis (EDA)

This is the core of our notebook, where we visualize and analyze:

### 4.1. ⚡ Overall Demand Over Time
- Total or average demand by hour/day
- Line plot of demand trends

### 4.2. 🕓 Hourly Patterns
- Mean demand by hour of the day
- Compare across months or seasons

### 4.3. 📆 Weekly & Monthly Trends
- Boxplots by day of week
- Monthly demand averages

### 4.4. 🌡️ Seasonal Patterns
- Compare summer vs winter demand profiles
- Heatmaps: hour vs day, hour vs month

## 5. 🔍 Key Insights

We summarize the most important insights from the EDA section:

- Demand peaks consistently at 7–9 PM, especially in summer.
- Weekends show significantly lower demand than weekdays (~X%).
- Winter shows a morning demand spike not seen in other seasons.
- [Other interesting insight here]

## 6. 📌 Conclusions & Next Steps

This exploratory analysis revealed consistent temporal demand patterns in Mexico's grid that may support better forecasting and operations planning.

### Next steps:
- Break down demand by geographic region (if data available)
- Correlate demand with temperature or weather patterns
- Build a simple forecasting model using Prophet or scikit-learn

## 🧠 Appendix

Any extra functions, alternative visualizations, failed experiments, or detailed technical notes can go here.

This keeps the main notebook clean and reader-friendly.